In [ ]:
import numpy as np
from scipy.constants import speed_of_light
import matplotlib.pyplot as plt
from matplotlib import colors
import itur_p2040 as p2040
import itur_p527 as p527
import circular_lossy_waveguide as pca
import aux_funcs

In [ ]:
# Consider pipe diameters from 10 mm (1e-2 m) to 10 000 mm (10 m)
pipe_diameters = np.logspace(-2, 1, 301)

# Consider frequencies from 0.1 GHz (1e8 Hz) to 1 000 GHz (1e12 Hz)
freqs = np.logspace(8, 12, 401)
freqs /= 1e9

In [ ]:
k_factor = 10
xv, yv = np.meshgrid(freqs, pipe_diameters, indexing='ij')

In [ ]:
check_value_out = np.zeros((np.size(freqs), np.size(pipe_diameters)))
check_value_in = np.zeros((np.size(freqs), np.size(pipe_diameters)))
check_value_circ_eh11 = np.zeros((np.size(freqs), np.size(pipe_diameters)))
check_value_circ_te01 = np.zeros((np.size(freqs), np.size(pipe_diameters)))
check_value_circ_te02 = np.zeros((np.size(freqs), np.size(pipe_diameters)))

In [ ]:
for i, freq in enumerate(freqs):
    for j, diameter in enumerate(pipe_diameters):
        check_value_out[i, j] = p2040.check_electrical_size_wavelength(freq, diameter, k_factor)
        check_value_in[i, j] = p2040.check_electrical_size_wavelength(freq, diameter/2*np.sqrt(2), k_factor)

In [ ]:
check_value_in_ma = np.ma.masked_where(check_value_in <= 1.0, check_value_in)
check_value_out_ma = np.ma.masked_where(check_value_out <= 1.0, check_value_out)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True)
fig.set_size_inches(10, 10/1.618)

axes[0].set_yscale('log')
axes[0].set_xscale('log')
axes[0].grid(which='major')
axes[0].set_xlim(np.min(freqs), np.max(freqs))
axes[0].set_ylim(np.min(pipe_diameters), np.max(pipe_diameters))
contour = axes[0].contourf(xv, yv, check_value_out_ma, cmap='cividis', norm=colors.LogNorm())

axes[1].set_yscale('log')
axes[1].set_xscale('log')
axes[1].grid(which='major')
axes[1].set_xlim(np.min(freqs), np.max(freqs))
axes[1].set_ylim(np.min(pipe_diameters), np.max(pipe_diameters))
contour = axes[1].contourf(xv, yv, check_value_in_ma, cmap='cividis', norm=colors.LogNorm())

axes[0].set_title('Inscribed circle approximation')
axes[1].set_title('Circumscribed circle approximation')
axes[1].set_xlabel('Frequency, (Hz)')

fig.tight_layout()

cbar = fig.colorbar(contour, ax=axes.ravel().tolist())
cbar.set_label('Check value')

fig.text(0, 0.5, 'Pipe diameter, (m)', va='center', rotation='vertical')

# fig.savefig('4.png', dpi=600, bbox_inches='tight')


In [ ]:
soil_temp = 10.0
soil_water_vol = 0.5
soil = p527.SOILS['silty_clay']
soil_permittivity = np.zeros(np.size(freqs), dtype=complex)

In [ ]:
for idx, freq in enumerate(freqs):
    foo = p527.soil_permittivity(freq, soil_temp, soil.p_sand, soil.p_clay, soil.p_silt, soil_water_vol)
    soil_permittivity[idx] = complex(foo.real, -foo.imag)

In [ ]:
for i, freq in enumerate(freqs):
    for j, diameter in enumerate(pipe_diameters):
        check_value_circ_eh11[i, j] = pca.check_electrical_size(freq, diameter, soil_permittivity[i], 1, 1, k_factor)
        check_value_circ_te01[i, j] = pca.check_electrical_size(freq, diameter, soil_permittivity[i], 0, 1, k_factor) 
        check_value_circ_te02[i, j] = pca.check_electrical_size(freq, diameter, soil_permittivity[i], 0, 2, k_factor)

In [ ]:
check_value_circ_eh11_ma = np.ma.masked_where(check_value_circ_eh11 <= 1.0, check_value_circ_eh11)
check_value_circ_te01_ma = np.ma.masked_where(check_value_circ_te01 <= 1.0, check_value_circ_te01)
check_value_circ_te02_ma = np.ma.masked_where(check_value_circ_te02 <= 1.0, check_value_circ_te02)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True)
fig.set_size_inches(10, 10/1.618)

axes[0].set_yscale('log')
axes[0].set_xscale('log')
axes[0].grid(which='major')
axes[0].set_xlim(np.min(freqs), np.max(freqs))
axes[0].set_ylim(np.min(pipe_diameters), np.max(pipe_diameters))
contour = axes[0].pcolormesh(xv, yv, check_value_circ_eh11_ma, cmap='cividis', norm=colors.LogNorm(vmin=1, vmax=1e3))

axes[1].set_yscale('log')
axes[1].set_xscale('log')
axes[1].grid(which='major')
axes[1].set_xlim(np.min(freqs), np.max(freqs))
axes[1].set_ylim(np.min(pipe_diameters), np.max(pipe_diameters))
contour = axes[1].pcolormesh(xv, yv, check_value_circ_te01_ma, cmap='cividis', norm=colors.LogNorm(vmin=1, vmax=1e3))

axes[2].set_yscale('log')
axes[2].set_xscale('log')
axes[2].grid(which='major')
axes[2].set_xlim(np.min(freqs), np.max(freqs))
axes[2].set_ylim(np.min(pipe_diameters), np.max(pipe_diameters))
contour = axes[2].pcolormesh(xv, yv, check_value_circ_te02_ma, cmap='cividis', norm=colors.LogNorm(vmin=1, vmax=1e3))

axes[0].set_title('EH11 mode')
axes[1].set_title('TE01 mode')
axes[2].set_title('TE02 mode')
axes[2].set_xlabel('Frequency, (Hz)')

fig.tight_layout()

cbar = fig.colorbar(contour, ax=axes.ravel().tolist())
cbar.set_label('Check value')

fig.text(0, 0.5, 'Pipe diameter, (m)', va='center', rotation='vertical')
fig.savefig('Silty Clay 2.png', dpi=600)